# Градиентный Бустинг

## 1. GradBoosting

Формат ввода
Датасет
onlinefoods.csv

Загрузить данные из датасета onlinefoods.csv
Закодировать целевую переменную с помощью LabelEncoder
Удалить столбцы "Output","Unnamed: 12", "Feedback"
Закодировать категориальные признаки с помощью LabelEncoder
Выполнить разбиение выборки с помощью train_test_split, train=80%, test=20%
Выполнить обучение GradientBoostingClassifier с параметрами по умолчанию
Вычислить recall, precision, f1, accuracy на тестовой подвыборке (сохраните в переменные recall, precision, f1, accuracy)
С помощью GridSearchCV настроить гиперпараметры для GradientBoostingClassifier: количество деревьев, скорость обучения, глубина дерева.
Вычислить recall, precision, f1, accuracy на тестовой подвыборке (сохраните в переменные tuned_recall, tuned_precision, tuned_f1, tuned_accuracy)
Условия прохождения теста:

tuned_recall >= 0.85
tuned_precision >= 0.87
tuned_f1 >= 0.86
tuned_accuracy >= 0.76

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

In [5]:
data = pd.read_csv('onlinefoods.csv')

y = data['Feedback']
le_tar = LabelEncoder()
y = le_tar.fit_transform(y)

In [6]:
drop_cols = ['Output', 'Unnamed: 12', 'Feedback']
X = data.drop(drop_cols, axis=1)

cat_cols = X.select_dtypes('object').columns

encoders = {}

In [7]:
for col in cat_cols:
    encoder = LabelEncoder()
    X[col] = encoder.fit_transform(X[col])
    encoders[col] = encoder

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=42)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [9]:
y_pred = gbc.predict(X_test)

recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [10]:
# Гиперпараметры
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7]
}

In [11]:
grid_search = GridSearchCV(gbc, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_gbc = grid_search.best_estimator_

y_pred_tuned = best_gbc.predict(X_test)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [12]:
tuned_recall = recall_score(y_test, y_pred_tuned)
tuned_precision = precision_score(y_test, y_pred_tuned)
tuned_f1 = f1_score(y_test, y_pred_tuned)
tuned_accuracy = accuracy_score(y_test, y_pred_tuned)

In [13]:
print (tuned_recall)
print (tuned_precision)
print (tuned_f1)
print (tuned_accuracy)

0.8208955223880597
0.8870967741935484
0.8527131782945736
0.7564102564102564


## 2. HistGradBoosting

Формат ввода
Датасет:
movie_metadata.csv

Целевая переменная: imdb_score

Загрузить данные из датасета movie_metadata.csv
Закодировать целевую переменную imdb_score с помощью LabelEncoder
Удалить столбцы 'movie_imdb_link', 'color', 'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'movie_title', 'plot_keywords', 'genres'
Выполнить бинаризацию целевой переменной [0, 4, 6, 8, 10]
Создать новый столбец как сумму признаков actor_2 и actor_3
Удалите избыточные столбцы ('actor_2_facebook_likes', 'actor_3_facebook_likes', 'cast_total_facebook_likes')
Обработайте пропуски
Разделите на подвыборки train и test
Обучите модель HistGradientBoostingClassifier с параметрами max_iter=100, learning_rate=0.09, max_depth=5
Вычислите метрику качества accuracy (accuracy)
Настройте гиперпараметры (max_iter, learning_rate, max_depth)
Вычислите метрику качества accuracy (final_accuracy)
Условия прохождения теста:

RocAUC >=0.85

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, Binarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [15]:
# Загрузка данных
data = pd.read_csv('movie_metadata.csv')

In [16]:
# Кодирование целевой переменной
label_encoder = LabelEncoder()
data['imdb_score'] = label_encoder.fit_transform(data['imdb_score'])

In [17]:
# Кодирование целевой переменной
label_encoder = LabelEncoder()
data['imdb_score'] = label_encoder.fit_transform(data['imdb_score'])

In [18]:
# Удаление ненужных столбцов
columns_to_drop = ['movie_imdb_link', 'color', 'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'movie_title', 'plot_keywords', 'genres']
data.drop(columns=columns_to_drop, inplace=True)

In [19]:
# Бинаризация целевой переменной
binarizer = Binarizer(threshold=6)
data['imdb_score'] = binarizer.fit_transform(data[['imdb_score']])

In [20]:
# Создание нового столбца как суммы признаков actor_2 и actor_3
data['actor_2_3_facebook_likes'] = data['actor_2_facebook_likes'] + data['actor_3_facebook_likes']

In [21]:
# Удаление избыточных столбцов
data.drop(columns=['actor_2_facebook_likes', 'actor_3_facebook_likes', 'cast_total_facebook_likes'], inplace=True)

In [22]:
# Разделение на числовые и категориальные признаки
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

In [23]:
# Создание пайплайна для обработки данных
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])


In [24]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # Устанавливаем sparse_output=False
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [25]:
# Применение пайплайна к данным
data_processed = preprocessor.fit_transform(data)

In [26]:
# Разделение на подвыборки train и test
X = data_processed
y = data['imdb_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Обучение модели HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(max_iter=100, learning_rate=0.09, max_depth=5)
model.fit(X_train, y_train)

HistGradientBoostingClassifier(learning_rate=0.09, max_depth=5)

In [28]:
# Вычисление метрики качества accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9910802775024777


In [29]:
# Настройка гиперпараметров
model_tuned = HistGradientBoostingClassifier(max_iter=150, learning_rate=0.1, max_depth=6)
model_tuned.fit(X_train, y_train)

HistGradientBoostingClassifier(max_depth=6, max_iter=150)

In [30]:
# Вычисление метрики качества accuracy после настройки
y_pred_tuned = model_tuned.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred_tuned)
print(f'Final Accuracy: {final_accuracy}')

Final Accuracy: 0.9910802775024777


In [31]:
# Вычисление метрики качества RocAUC
y_pred_proba = model_tuned.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'RocAUC: {roc_auc}')

RocAUC: 0.7822222222222223


In [32]:
# Проверка условия прохождения теста
if roc_auc >= 0.85:
    print("Тест пройден: RocAUC >= 0.85")
else:
    print("Тест не пройден: RocAUC < 0.85")

Тест не пройден: RocAUC < 0.85


In [33]:
# Обучение модели HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(max_iter=100, learning_rate=0.09, max_depth=5)
model.fit(X_train, y_train)

HistGradientBoostingClassifier(learning_rate=0.09, max_depth=5)

In [34]:
# Вычисление метрики качества accuracy после настройки
y_pred_tuned = model_tuned.predict(X_test)
final_accuracy = accuracy_score(y_test, y_pred_tuned)
print(f'Final Accuracy: {final_accuracy}')

Final Accuracy: 0.9910802775024777


In [36]:
# Вычисление метрики качества RocAUC
y_pred_proba = model_tuned.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'RocAUC: {roc_auc}')

RocAUC: 0.7822222222222223


In [37]:
# Проверка условия прохождения теста
if roc_auc >= 0.85:
    print("Тест пройден: RocAUC >= 0.85")
else:
    print("Тест не пройден: RocAUC < 0.85")

Тест не пройден: RocAUC < 0.85
